In [1]:
import cv2
import albumentations as A
from matplotlib import pyplot as plt
import plotly.express as px
import json
import imageio
import imgaug as ia
import imgaug.augmenters as iaa
import numpy as np
from imgaug.augmentables.polys import Polygon, PolygonsOnImage

In [27]:
class AugmentateImages:
    def __init__(self, img_paths, json_paths, json_export_path, check_augmentation=False):
        self.img_paths = img_paths
        self.json_paths = json_paths
        self.json_export_path = json_export_path
        self.is_visualize = check_augmentation
        self.start_id = len(self.img_paths) + 1

    def load_image(self):
        img = imageio.imread(self.img_path)
        img = ia.imresize_single_image(img, 1.0)
        return img

    def load_json(self):
        with open(self.json_path, "r") as f:
            data = json.load(f)
        return data

    def get_annotation(self):
        f = self.load_json()
        return np.asarray(f["shapes"][0]["points"])

    def dump_annotations(self, dic, img, img_path, id, segmentation, bbox):
        # Initialize dictionary if empty
        if len(dic) == 0:
            dic = {}
            dic["images"] = []
            dic["categories"] = [{"supercategory": "ramp", "id": 1, "name": "ramp"}]
            dic["annotations"] = []

        # Create the new entry
        file_info = {"height": img.shape[0], "width": img.shape[1], "id": id, "file_name": img_path}
        dic["images"].append(file_info)
        annotation_info = {
            "iscrowd": 0,
            "image_id": id,
            "bbox": bbox,
            "segmentation": segmentation,
            "category_id": 1,
            "id": id,
            "area": 589824,
        }
        dic["annotations"].append(annotation_info)

        # Add to json file
        with open(self.json_export_path, "w") as f:
            json.dump(dic, f, indent=4)
        return dic

    def dump_image(self, img):
        aug_path = self.img_path[:-4] + "_aug.jpg"
        cv2.imwrite(aug_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
        return aug_path

    def correct_format(self, poly, box):
        # Convert polygon to list
        poly_flat = [poly.to_xy_array().flatten().tolist()]
        # Flatten bounding box
        box_flat = box.to_xy_array().flatten().tolist()
        # Convert box to correct format
        box_flat = self.convert(box_flat)
        return poly_flat, box_flat

    def convert(self, box):
        """Convert box coordinates from ? to x0, y0, w, h"""
        x_min = min(box[0::2])
        y_min = min(box[1::2])
        w = max(box[0::2]) - x_min
        h = max(box[1::2]) - y_min
        return x_min, y_min, w, h

    def aug(self, image, polygons):
        # Get correct polygon format
        poly_oi = ia.PolygonsOnImage([polygons], shape=image.shape)
        box_oi = ia.BoundingBoxesOnImage([polygons.to_bounding_box()], shape=image.shape)

        # Create augmentation pipeline
        aug = iaa.Sequential(
            [
                iaa.Fliplr(0.5),
                iaa.AdditiveGaussianNoise(scale=3),
                iaa.MultiplyBrightness(mul=(0.5, 1.5)),
                # iaa.Crop(percent=(0, 0.2)),
            ]
        )

        # Run augmentation
        image_aug, poly_aug, box_aug = aug(image=image, polygons=poly_oi, bounding_boxes=box_oi)
        return image_aug, poly_aug, box_aug

    def check_augmentation(self, img, polygons, box):
        image = np.copy(img)
        image = polygons.draw_on_image(image, alpha_face=0.2, size_points=7)
        image = box.draw_on_image(image, size=1)
        px.imshow(image).show()

    def main(self):
        dic = {}
        for i, _ in enumerate(self.img_paths):
            # Select single image and annotation
            self.img_path = self.img_paths[i]
            self.json_path = self.json_paths[i]

            # Load the image
            img = self.load_image()
            # Load the 4 coordinates of the polygon
            poly = self.get_annotation()
            # Convert to polygon format
            poly = Polygon(poly)

            # Run the augmentation
            img_aug, poly_aug, box_aug = self.aug(img, poly)
            # Check the augmentation
            if self.is_visualize:
                self.check_augmentation(img_aug, poly_aug, box_aug)
            # Get correct format for json
            poly_aug, box_aug = self.correct_format(poly_aug, box_aug)

            # Save the image
            img_aug_path = self.dump_image(img_aug)
            # Save the annotations
            idx = self.start_id + i
            dic = self.dump_annotations(dic, img, img_aug_path, idx, poly_aug, box_aug)
        return img


In [28]:
path = "/home/user/rosbags/final/object_detection"
import os
file_list = os.listdir(path)
# Get all images
img_paths = sorted([os.path.join(path, file) for file in file_list if file.endswith(".jpg") and "aug" not in file])
# Get all json files
json_paths = sorted([os.path.join(path, file) for file in file_list if file.endswith(".json")])

json_export_path = 'augmentated.json'
# AI = AugmentateImages(img_paths, json_paths, json_export_path, True)
AI = AugmentateImages(img_paths, json_paths, json_export_path)
img = AI.main()
